<a href="https://colab.research.google.com/github/syuan0525/project-team/blob/main/modelgenerator1109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.applications.mobilenet import MobileNet
mobilenet = MobileNet(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet'
)
mobilenet.summary()

17235968/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
mobilenet.layers

In [4]:
for l in mobilenet.layers:
  l.trainable = False  # cnn 已經是訓練好的了，所以先凍住cnn的參數 ，只訓練後面的mlp的參數 # 要在 compile 前設

layers = [
          BatchNormalization(),
          GlobalAveragePooling2D(),
          Dropout(0.25),
          # Dense(256, activation="relu"),
          Dense(11, activation="softmax")
]
model = Sequential(mobilenet.layers + layers) 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pw_1 (Conv2D)           (None, 112, 112, 64)      2

In [5]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(
        loss=SparseCategoricalCrossentropy(),
        optimizer="adam",
        metrics=["acc"]
)

In [6]:
import glob
import os
os.chdir("/content/drive/MyDrive/Tebame_第四組_機車辨識")

In [7]:
s2 = glob.glob("./裁剪照片(不含不用裁的)/GOGORO_S2_裁/*.jpg") 

gp = glob.glob("./裁剪照片(不含不用裁的)/KYMCO_GP_125_裁/*") 

krv = glob.glob("./裁剪照片(不含不用裁的)/KYMCO_KRV_裁/*")

fiddle = glob.glob("./裁剪照片(不含不用裁的)/SYM_FIDDLE_125_裁/*") 

sr = glob.glob("./裁剪照片(不含不用裁的)/SYM_JET_SR_125_裁/*")

vivo = glob.glob("./裁剪照片(不含不用裁的)/SYM_活力_125_裁/*")

famous = glob.glob("./裁剪照片(不含不用裁的)/KYMCO_新名流125_裁/*")

drg = glob.glob("./裁剪照片(不含不用裁的)/SYM_DRG_裁/*") 

duke = glob.glob("./裁剪照片(不含不用裁的)/SYM_新迪爵_125 _裁/*")

limi = glob.glob("./裁剪照片(不含不用裁的)/YAMAHA_LIMI_125_已裁減/*")

gryphus6 = glob.glob("./裁剪照片(不含不用裁的)/YAMAHA_六代勁戰_125_裁/*")

In [8]:
print("s2 共:", len(s2))
print("gp 共:", len(gp))
print("krv 共:", len(krv))
print("fiddle 共:", len(fiddle))
print("sr 共:", len(sr))
print("vivo 共:", len(vivo))
print("famous 共:", len(famous))
print("drg 共:", len(drg))
print("duke 共:", len(duke))
print("limi 共:", len(limi))
print("gryphus6 共:", len(gryphus6))

s2 共: 349
gp 共: 265
krv 共: 331
fiddle 共: 649
sr 共: 219
vivo 共: 354
famous 共: 148
drg 共: 418
duke 共: 456
limi 共: 322
gryphus6 共: 541


In [9]:
import tensorflow as tf
import cv2
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa
import imgaug as ia
from tensorflow.keras.applications.mobilenet import preprocess_input 

In [10]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, img_paths, batch_size, img_size, shuffle=True, aug=False):
        self.img_paths = img_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.aug = aug

        self.indexes = np.arange(len(self.img_paths)) # [0, 1, 2, 3, 4, 5]
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.img_paths) / self.batch_size)) # batches per epoch

    def __getitem__(self, index):
        # Generate indexes of the batch
        idxs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
        batch_img_paths = [self.img_paths[i] for i in idxs]

        # Generates data containing batch_size samples
        x = np.empty((len(batch_img_paths), self.img_size, self.img_size, 3), dtype=np.float32)
        y = np.empty((len(batch_img_paths), 1), dtype=np.float32)
        # y = np.array(df["ans"])

        for i, img_path in enumerate(batch_img_paths):
            # read img
            # img = cv2.imread(img_path)
            # img = cv2.resize(img, (self.img_size, self.img_size))
            # img = img / 255.

            img = Image.open(img_path).resize((self.img_size, self.img_size)).convert("RGB")
            img = np.array(img)
            img = preprocess_input(img)

            if self.aug:
              seq = iaa.Sequential([
                      iaa.Fliplr(0.5), # 50% horizontal flip
                      iaa.Flipud(0), # 50% vertical flip
                      iaa.Affine(
                      rotate=(-45, 45), # random rotate -45 ~ +45 degree
                      shear=(-16, 16), # random shear -16 ~ +16 degree
                      scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale x, y: 80%~120%,
                      mode='constant', # ia.ALL, 'constant','edge', 'symmetric', 'reflect', 'wrap'
                      cval=0,
                        ),
                      ])
              img_aug = seq.augment_image(img)
              img = img_aug # Augmentation

            # read class label
            cls = img_path.split('/')[-2]
            if "S2" in cls:
                cls = 0
            elif "GP" in cls:
                cls = 1
            elif "KRV" in cls:
                cls = 2
            elif "FIDDLE" in cls:
                cls = 3
            elif "SR" in cls:
                cls = 4
            elif "活力" in cls:
                cls = 5
            elif "新名流" in cls:
                cls = 6
            elif "DRG" in cls:
                cls = 7
            elif "新迪爵" in cls:
                cls = 8
            elif "LIMI" in cls:
                cls = 9
            elif "六代" in cls:
                cls = 10      
            x[i] = img
            y[i] = cls
            
        return x, y

    def on_epoch_end(self):
        # Updates indexes after each epoch
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [11]:
img_paths = s2 + gp + krv + fiddle + sr + vivo + famous + drg + duke + limi + gryphus6
len(img_paths)

4052

In [12]:
from sklearn.model_selection import train_test_split
train_img_paths, val_img_paths = train_test_split(img_paths, test_size=0.1)
len(train_img_paths), len(val_img_paths)

(3646, 406)

In [13]:
train_gen = DataGenerator(train_img_paths, 32, 224, shuffle=True, aug=True)
val_gen = DataGenerator(val_img_paths, 32, 224, shuffle=False, aug=False)

In [14]:
train_gen.indexes

array([3603,  447, 1284, ...,  843, 3630,  446])

In [15]:
len(train_gen)

114

In [16]:
imgs, labels = train_gen[2]

In [17]:
imgs.shape, labels.shape

((32, 224, 224, 3), (32, 1))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
             EarlyStopping(patience=10, restore_best_weights=True),
             ModelCheckpoint("/content/1109.h5", save_best_only=True)
]

model.fit(
    train_gen,
    steps_per_epoch=10,
    epochs=300,
    validation_data=val_gen,
    validation_steps=2, 
    callbacks=callbacks
                    )

Epoch 1/300
10/10 [==============================] - 144s 11s/step - loss: 2.5644 - acc: 0.0906 - val_loss: 2.5784 - val_acc: 0.0156
Epoch 2/300
10/10 [==============================] - 90s 9s/step - loss: 2.2258 - acc: 0.0969 - val_loss: 2.1975 - val_acc: 0.0781
Epoch 3/300
10/10 [==============================] - 84s 8s/step - loss: 2.0104 - acc: 0.1187 - val_loss: 1.9143 - val_acc: 0.1094
Epoch 4/300
 6/10 [=================>............] - ETA: 29s - loss: 1.8889 - acc: 0.0833